To see the potential benefits of using interest and emphasis, consider the four-state
Markov reward process shown below:

+1     +1     +1     +1  
vπ = 4  vπ = 3  vπ = 2  vπ = 1  
i = 1  i = 0  i = 0  i = 0  
w₁     w₁     w₂     w₂  

Episodes start in the leftmost state, then transition one state to the right, with a
reward of +1 on each step until the terminal state is reached. The true value of
the first state is thus 4, of the second state 3, and so on as shown below each state.
These are the true values; the estimated values can only approximate these because
they are constrained by the parameterization. There are two components to the
parameter vector **w = (w₁, w₂)ᵀ**, and the parameterization is as written inside
each state. The estimated values of the first two states are given by **w₁** alone and
thus must be the same even though their true values are different. Similarly, the
estimated values of the third and fourth states are given by **w₂** alone and must be
the same even though their true values are different. Suppose that we are interested
in accurately valuing only the leftmost state; we assign it an interest of 1 while all
the other states are assigned an interest of 0, as indicated above the states.

---

First consider applying **gradient Monte Carlo algorithms** to this problem. The
algorithms presented earlier in this chapter that do not take into account interest
and emphasis (in (9.7) and the box on page 202) will converge (for decreasing step
sizes) to the parameter vector **w₁ = (3.5, 1.5)**, which gives the first state—the only
one we are interested in—a value of 3.5 (i.e., intermediate between the true values
of the first and second states). The methods presented in this section that do use
interest and emphasis, on the other hand, will learn the value of the first state
exactly correctly; **w₁** will converge to 4 while **w₂** will never be updated because the
emphasis is zero in all states save the leftmost.

---

Now consider applying **two-step semi-gradient TD methods**. The methods from
earlier in this chapter without interest and emphasis (in (9.15) and (9.16) and
the box on page 209) will again converge to **w₁ = (3.5, 1.5)**, while the methods
with interest and emphasis converge to **w₁ = (4, 2)**. The latter produces the
exactly correct values for the first state and for the third state (which the first state
bootstraps from) while never making any updates corresponding to the second or
fourth states.


In [ ]:
import numpy as np

states = [0, 1, 2, 3, 4]
interest = [1, 0, 0, 0, 0]
start_state = 0
terminal_states = [4]

TRUE_VALUES = [4, 3, 2, 1, 0]

def get_reward_and_next_state(state, action='right'):
    if action == 'right':
        if state == 4:
            return 0, 4
        else:
            return 1, state + 1
    elif action == 'left':
        if state == 0:
            return 0, 0
        else:
            return 1, state - 1
    else:
        raise ValueError(f"Invalid action: {action}")

def feature_vector(state):
    if state == 4:
        return np.array([0.0, 0.0], dtype=np.float64)
    if state == 0 or state == 1:
        return np.array([1.0, 0.0], dtype=np.float64)
    if state == 2 or state == 3:
        return np.array([0.0, 1.0], dtype=np.float64)
    raise ValueError(f"Invalid state: {state}")

def value_function(w, state):
    if state in terminal_states:
        return 0.0
    return float(np.dot(w, feature_vector(state)))

def generate_episode():
    state = start_state
    episode = []
    while state not in terminal_states:
        reward, next_state = get_reward_and_next_state(state, 'right')
        episode.append((state, reward))
        state = next_state
    return episode

def gradient_mc_update(w, num_episodes, alpha, gamma=1.0):
    for _ in range(num_episodes):
        episode = generate_episode()  # list of (state, reward)
        G = 0.0
        M = 0.0
        for i, (state, reward) in enumerate(reversed(episode)):
            G = reward + gamma * G
            v_hat = value_function(w, state)
            grad = feature_vector(state)
            w += alpha * (G - v_hat) * grad * interest[state]
    return w


w = np.zeros(2, dtype=np.float64)
w = gradient_mc_update(w, 10000, 0.001)
print(w)
estimates = [value_function(w, s) for s in states]
print(estimates)

[3.99981931 0.        ]
[3.999819306616098, 3.999819306616098, 0.0, 0.0, 0.0]
